In [1]:
import sys
sys.path.append("../src")
from data_explore import (
 load_fi_data, explore_impact_mechanics, summarize_pillars,load_reference_codes,validate_records,analyze_impact_links,get_record_counts,get_temporal_coverage,list_unique_indicators
)
import pandas as pd

In [2]:
# Load the Excel file
data = load_fi_data("../data/raw/ethiopia_fi_unified_data.xlsx")
df_main = data['unified_data']
df_impacts = data['impact_links']

# Segment record types in Sheet 1
observations = df_main[df_main['record_type'] == 'observation']
events = df_main[df_main['record_type'] == 'event']
targets = df_main[df_main['record_type'] == 'target']

print(f"Main Data: {len(observations)} Obs, {len(events)} Events, {len(targets)} Targets")
print(f"Impact Links: {len(df_impacts)} modeled relationships found in 'Impact_sheet'")

Main Data: 30 Obs, 10 Events, 3 Targets
Impact Links: 14 modeled relationships found in 'Impact_sheet'


In [3]:
# Map events to their indicators via parent_id in Impact_sheet
mapping = explore_impact_mechanics(data)

# Analyze how 'Product Launches' affect indicators
print("Sample Event-to-Indicator Mapping:")
display(mapping[['parent_id', 'indicator_event', 'indicator_link', 'impact_magnitude']].head())

Sample Event-to-Indicator Mapping:


,parent_id,indicator_event,indicator_link,impact_magnitude
0,EVT_0001,Telebirr Launch,Telebirr effect on Account Ownership,high
1,EVT_0001,Telebirr Launch,Telebirr effect on Telebirr Users,high
2,EVT_0001,Telebirr Launch,Telebirr effect on P2P Transactions,high
3,EVT_0002,Safaricom Ethiopia Commercial Launch,Safaricom effect on 4G Coverage,medium
4,EVT_0002,Safaricom Ethiopia Commercial Launch,Safaricom effect on Data Affordability,medium


In [4]:
# Load Reference Codes
ref_codes = load_reference_codes("../data/raw/reference_codes.xlsx")

# Run Validation
validation_results = validate_records(df_main, ref_codes)

if validation_results['invalid_indicator_count'] > 0:
    print(f"⚠️ Warning: Found {validation_results['invalid_indicator_count']} records with invalid indicator codes.")
    print(f"Check these codes: {validation_results['flagged_indicator_codes']}")
else:
    print("✅ Schema Check Passed: All codes align with reference_codes.csv")

⚠️ Warning: Found 43 records with invalid indicator codes.
Check these codes: ['ACC_OWNERSHIP', 'ACC_MM_ACCOUNT', 'ACC_4G_COV', 'ACC_MOBILE_PEN', 'ACC_FAYDA', 'USG_P2P_COUNT', 'USG_P2P_VALUE', 'USG_ATM_COUNT', 'USG_ATM_VALUE', 'USG_CROSSOVER', 'USG_TELEBIRR_USERS', 'USG_TELEBIRR_VALUE', 'USG_MPESA_USERS', 'USG_MPESA_ACTIVE', 'USG_ACTIVE_RATE', 'AFF_DATA_INCOME', 'GEN_GAP_ACC', 'GEN_MM_SHARE', 'GEN_GAP_MOBILE', 'EVT_TELEBIRR', 'EVT_SAFARICOM', 'EVT_MPESA', 'EVT_FAYDA', 'EVT_FX_REFORM', 'EVT_CROSSOVER', 'EVT_MPESA_INTEROP', 'EVT_ETHIOPAY', 'EVT_NFIS2', 'EVT_SAFCOM_PRICE']


Count Records by Dimensions

In [5]:
counts = get_record_counts(df_main)
for dim, val in counts.items():
    print(f"--- {dim.upper()} ---")
    print(val)

--- RECORD_TYPE ---
{'observation': 30, 'event': 10, 'target': 3}
--- PILLAR ---
{'ACCESS': 16, 'USAGE': 11, nan: 10, 'GENDER': 5, 'AFFORDABILITY': 1}
--- SOURCE_TYPE ---
{'operator': 15, 'survey': 10, 'regulator': 7, 'research': 4, 'policy': 3, 'calculated': 2, 'news': 2}
--- CONFIDENCE ---
{'high': 40, 'medium': 3}


Temporal Range and Indicators

In [6]:
start, end = get_temporal_coverage(df_main)
print(f"Data Timeline: {start.year} to {end.year}")

indicator_summary = list_unique_indicators(df_main)
display(indicator_summary)

Data Timeline: 2014 to 2025


,record_count,latest_value,unit
indicator_code,,,
ACC_4G_COV,2,7.080000e+01,%
ACC_FAYDA,4,9.000000e+07,people
ACC_MM_ACCOUNT,2,9.450000e+00,%
ACC_MOBILE_PEN,1,6.140000e+01,%
ACC_OWNERSHIP,7,7.000000e+01,%
AFF_DATA_INCOME,1,2.000000e+00,% of GNI
EVT_CROSSOVER,1,NaN,None
EVT_ETHIOPAY,1,NaN,None
EVT_FAYDA,1,NaN,None


Catalog Events and Dates

In [7]:
events_catalog = df_main[df_main['record_type'] == 'event'][
    ['observation_date', 'indicator', 'notes', 'source_name']
].sort_values('observation_date')

print("Major Cataloged Events:")
display(events_catalog)

Major Cataloged Events:


,observation_date,indicator,notes,source_name
33,2021-05-17,Telebirr Launch,NaN,Ethio Telecom
41,2021-09-01,NFIS-II Strategy Launch,NaN,NBE
34,2022-08-01,Safaricom Ethiopia Commercial Launch,NaN,News
35,2023-08-01,M-Pesa Ethiopia Launch,NaN,Safaricom
36,2024-01-01,Fayda Digital ID Program Rollout,NaN,NIDP
37,2024-07-29,Foreign Exchange Liberalization,NaN,NBE
38,2024-10-01,P2P Transaction Count Surpasses ATM,NaN,EthSwitch
39,2025-10-27,M-Pesa EthSwitch Integration,NaN,EthSwitch
42,2025-12-15,Safaricom Ethiopia Price Increase,NaN,News
40,2025-12-18,EthioPay Instant Payment System Launch,NaN,NBE/EthSwitch


Review Impact Relationships

In [8]:
# Using the function we built in Task 1
impact_summary = df_impacts.groupby(['impact_direction', 'impact_magnitude']).size().unstack(fill_value=0)

print("Distribution of Modeled Impacts:")
display(impact_summary)

# Example: View relationships with a 'High' impact magnitude
high_impacts = df_impacts[df_impacts['impact_magnitude'] == 'high']
display(high_impacts[['parent_id', 'indicator', 'impact_direction', 'lag_months']])

Distribution of Modeled Impacts:


impact_magnitude,high,low,medium
impact_direction,,,
decrease,0,0,2
increase,5,1,6


,parent_id,indicator,impact_direction,lag_months
0,EVT_0001,Telebirr effect on Account Ownership,increase,12
1,EVT_0001,Telebirr effect on Telebirr Users,increase,3
2,EVT_0001,Telebirr effect on P2P Transactions,increase,6
5,EVT_0003,M-Pesa effect on M-Pesa Users,increase,3
9,EVT_0005,FX Reform effect on Data Affordability,increase,3
